# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv('/content/drive/MyDrive/dataset/dataset_clustering.csv')

# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [4]:
df.head()

,TransactionType,Channel,TransactionAmount,CustomerAge,TransactionDuration,AccountBalance,Cluster
0,Debit,ATM,14.09,70,81.0,5112.21,3
1,Debit,ATM,376.24,68,141.0,13758.91,3
2,Debit,Online,126.29,19,56.0,1122.35,0
3,Debit,Online,184.50,26,25.0,8569.06,0
4,Credit,Online,13.45,26,198.0,7429.40,4


In [5]:
# Buat instance LabelEncoder
label_encoder = LabelEncoder()

# List kolom kategorikal yang perlu di-encode
categorical_columns = ['TransactionType', 'Channel']

# Encode kolom kategorikal
for column in categorical_columns:
    df[column] = label_encoder.fit_transform(df[column])

# Tampilkan DataFrame untuk memastikan encoding telah diterapkan
df.head()

,TransactionType,Channel,TransactionAmount,CustomerAge,TransactionDuration,AccountBalance,Cluster
0,1,0,14.09,70,81.0,5112.21,3
1,1,0,376.24,68,141.0,13758.91,3
2,1,2,126.29,19,56.0,1122.35,0
3,1,2,184.50,26,25.0,8569.06,0
4,0,2,13.45,26,198.0,7429.40,4


In [6]:
# Buat instance MinMaxScaler
scaler = MinMaxScaler()

# Normalisasi semua kolom numerik
numeric_columns = ['TransactionAmount', 'CustomerAge', 'TransactionDuration', 'AccountBalance']
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

df.head()

,TransactionType,Channel,TransactionAmount,CustomerAge,TransactionDuration,AccountBalance,Cluster
0,1,0,0.015317,0.838710,0.244828,0.336832,3
1,1,0,0.416400,0.806452,0.451724,0.918055,3
2,1,2,0.139579,0.016129,0.158621,0.068637,0
3,1,2,0.204047,0.129032,0.051724,0.569198,0
4,0,2,0.014608,0.129032,0.648276,0.492591,4


In [7]:
# Pisahkan fitur (X) dan target (y)
X = df.drop(columns=['Cluster'])
y = df['Cluster']

# Split data menjadi set pelatihan dan set uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tampilkan bentuk set pelatihan dan set uji untuk memastikan split
print(f"Training set shape: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"Test set shape: X_test={X_test.shape}, y_test={y_test.shape}")

Training set shape: X_train=(1919, 6), y_train=(1919,)
Test set shape: X_test=(480, 6), y_test=(480,)


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [8]:
# Definisikan setiap klasifikasi secara terpisah
knn = KNeighborsClassifier().fit(X_train, y_train)
dt = DecisionTreeClassifier().fit(X_train, y_train)

print("Model training selesai.")

Model training selesai.


Tulis narasi atau penjelasan algoritma yang Anda gunakan.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [9]:
# Fungsi untuk mengevaluasi dan mengembalikan hasil sebagai kamus
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)

    results = {
        'Confusion Matrix': cm,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, average='macro'),
        'Recall': recall_score(y_test, y_pred, average='macro'),
        'F1-Score': f1_score(y_test, y_pred, average='macro')
    }

    return results

# Mengevaluasi setiap model dan mengumpulkan hasilnya
results = {
    'K-Nearest Neighbors (KNN)': evaluate_model(knn, X_test, y_test),
    'Decision Tree (DT)': evaluate_model(dt, X_test, y_test)
}

# Buat DataFrame untuk meringkas hasil
summary_df = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])

# Isi DataFrame dengan hasil
rows = []
for model_name, metrics in results.items():
    rows.append({
        'Model': model_name,
        'Accuracy': metrics['Accuracy'],
        'Precision': metrics['Precision'],
        'Recall': metrics['Recall'],
        'F1-Score': metrics['F1-Score']
    })

# Konversi daftar kamus ke DataFrame
summary_df = pd.DataFrame(rows)

# Tampilkan DataFrame
print(summary_df)

                       Model  Accuracy  Precision  Recall  F1-Score
0  K-Nearest Neighbors (KNN)       1.0        1.0     1.0       1.0
1         Decision Tree (DT)       1.0        1.0     1.0       1.0


Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

**Analisis dan Perbandingan**
1. Akurasi: Kedua algoritma memiliki akurasi 100%, yang menunjukkan bahwa model dapat mengklasifikasikan data dengan sempurna pada set pengujian.

2. Precision, Recall, dan F1-Score: Semua metrik juga menunjukkan nilai sempurna (1.0), yang mengindikasikan bahwa tidak ada kesalahan klasifikasi dalam pengujian.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [10]:
# Hyperparameter tuning untuk KNN
knn_param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

knn_grid = GridSearchCV(KNeighborsClassifier(), knn_param_grid, cv=5, scoring='accuracy', n_jobs=-1)
knn_grid.fit(X_train, y_train)

best_knn = knn_grid.best_estimator_
y_pred = best_knn.predict(X_test)

# Menampilkan hasil terbaik
print("Best KNN parameters:", knn_grid.best_params_)

Best KNN parameters: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'uniform'}


In [11]:
# Definisi hyperparameter untuk Decision Tree
dt_param_grid = {
    'max_depth': [None, 5, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random']
}

# GridSearchCV untuk Decision Tree
dt_grid = GridSearchCV(DecisionTreeClassifier(), dt_param_grid, cv=5, scoring='accuracy', n_jobs=-1)
dt_grid.fit(X_train, y_train)

best_dt = dt_grid.best_estimator_
y_pred = best_dt.predict(X_test)

# Menampilkan hasil terbaik
print("Best Decision Tree parameters:", dt_grid.best_params_)

Best Decision Tree parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'best'}


## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [12]:
# Fungsi untuk mengevaluasi dan mengembalikan hasil sebagai kamus
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)

    results = {
        'Confusion Matrix': cm,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, average='macro'),
        'Recall': recall_score(y_test, y_pred, average='macro'),
        'F1-Score': f1_score(y_test, y_pred, average='macro')
    }

    return results

# Mengevaluasi setiap model dan mengumpulkan hasilnya
results = {
    'K-Nearest Neighbors (KNN)': evaluate_model(best_knn, X_test, y_test),
    'Random Forest (RF)': evaluate_model(best_dt, X_test, y_test),
}

# Buat DataFrame untuk meringkas hasil
summary_df = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])

# Isi DataFrame dengan hasil
rows = []
for model_name, metrics in results.items():
    rows.append({
        'Model': model_name,
        'Accuracy': metrics['Accuracy'],
        'Precision': metrics['Precision'],
        'Recall': metrics['Recall'],
        'F1-Score': metrics['F1-Score']
    })

# Konversi daftar kamus ke DataFrame
summary_df = pd.DataFrame(rows)

# Tampilkan DataFrame
print(summary_df)

                       Model  Accuracy  Precision  Recall  F1-Score
0  K-Nearest Neighbors (KNN)       1.0        1.0     1.0       1.0
1         Random Forest (RF)       1.0        1.0     1.0       1.0


**Analisis**
- Karena sebelumnya sempurna, yaitu 1.0, maka setelah dilakukan Hyperparameter tidak ada kenaikan. Kedua algoritma memiliki akurasi 100%, yang menunjukkan bahwa model dapat mengklasifikasikan data dengan sempurna pada set pengujian.

## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Perbandingan hasil evaluasi sebelum dan setelah tuning.
  - Setelah dilakukan tuning, tidak ada perubahan terhadap data. Hal ini mungkin disebabkan karena akurasi pada algoritma KNN dan DecisionTree yaitu 100%.
2. Identifikasi kelemahan model.
  - Model kali ini yang saya lakukan bisa jadi mengalami overfitting. Hal ini disebabkan karena akurasi yang sempurna terdapat model. Selain itu, dataset yang digunakan mungkin terlalu kecil atau kurang beragam, sehingga model tidak dapat belajar pola yang lebih general.
3. Rekomendasi tindakan lanjutan.
  - Jika menggunakan KNN, pastikan pemilihan jumlah tetangga (k) tidak terlalu kecil, karena nilai k yang sangat kecil dapat menyebabkan overfitting.
  - Melakukan penanganan overfitting atau underfitting.
  - Jika memungkinkan, kumpulkan lebih banyak data dengan variasi yang lebih luas untuk membantu model belajar pola yang lebih umum, bukan hanya yang spesifik pada dataset saat ini.